<a href="https://colab.research.google.com/github/Chien-noir/AMLS_22-23_SN18001485/blob/main/ResNet_modified.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import matplotlib.pyplot as plt
import glob
import PIL
import numpy as np
import tensorflow as tf
from tensorflow.keras.layers import Input
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import Dense,BatchNormalization,MaxPooling2D,Conv2D,ZeroPadding2D,Activation,AveragePooling2D, Flatten 
from tensorflow.keras.models import Model
from tensorflow.keras.preprocessing import image
import tensorflow.keras.backend as K
from tensorflow.keras.applications.imagenet_utils import preprocess_input, decode_predictions
from tensorflow.keras import layers
import os
import cv2
import dlib

# **Get labels**

In [ ]:
# PATH TO ALL IMAGES

basedir = '/content/drive/MyDrive/Colab Notebooks/dataset_AMLS_22-23'
images_dir = os.path.join(basedir,'cartoon_set')
images_dir = os.path.join(images_dir,'img')
labels_filename = 'labels.csv'

#image path and read and open all images

image_paths = [os.path.join(images_dir, l) for l in os.listdir(images_dir)]
target_size = None
labels_file = open(os.path.join('/content/drive/MyDrive/Colab Notebooks/dataset_AMLS_22-23/cartoon_set', labels_filename), 'r')
lines = labels_file.readlines()

#set the switch for accessing the label
eye_color= 1
face_shape = 2

labels = [int(line.split('\t')[face_shape]) for line in lines[1:]]
print(labels)
if os.path.isdir(images_dir):
    all_features = []
    all_labels = []
    for img_path in image_paths:
        file_name= img_path.split('.')[0].split('/')[-1]
        file_name = int(file_name)
        all_labels.append(labels[file_name])

labels = np.array(all_labels)  

# **Get graph data**

In [ ]:
target_size = (100, 100) #grpah target size
path = '/content/drive/MyDrive/Colab Notebooks/dataset_AMLS_22-23/cartoon_set/img'
filename = os.listdir(path)
input = []  #create an empty array
label_set = []
counter = 0
counting = 0
num_limit = 2000

image_data = []  #create a array for sotring the data 
for file in filename:
  counter = counter + 1
  counting = counting +1
  inputImg = PIL.Image.open(os.path.join(path,file)).convert('RGB')
  name_label = int((os.path.join(path,file)).split('.')[0].split('/')[-1])
  label = labels[name_label]
  label_set.append(labels[label])
  input_array= np.asarray(inputImg)

  #resize the arraty to have lower resulution, which is set by traget size
  input_array= cv2.resize(input_array,target_size)
  input.append(input_array)
  #print the process
  if counter %100 ==0:
    print('process:',counter)

  #break when the data reach the limitation
  if counting == num_limit:
    break

spl = int(0.8*num_limit)
train_input = input[:spl]
test_input = input[spl:]
train_label = label_set[:spl]
test_label = label_set[spl:]
input_tensor = tf.convert_to_tensor(train_input)               
input = np.array(input.append(input_array))
label_set = label_set
##convert the array into the tensor
#input_tensor = tf.convert_to_tensor(input)
label_tensor= tf.convert_to_tensor(train_label)

test_input_tensor = tf.convert_to_tensor(test_input)  
test_label_tensor= tf.convert_to_tensor(test_label)


In [ ]:
for element in input_tensor:
  print(element)

# **Test if the graph has been extracted**

In [ ]:
print(label_set)
print(input)
print('shape of the array',input.size)
for element in input_tensor:
  print(element)
  

this is the code for getting the data refered from online

In [ ]:
#img_path = glob.glob('foods/train/*/*.jpg')
img_path = glob.glob('/content/drive/MyDrive/Colab Notebooks/dataset_AMLS_22-23/cartoon_set/img1/*.png')
print(img_path)
#labels = [img.split('/')[8] for img in img_path]
#labels = [img.split('/')[8].split('.')[0] for img in img_path]
# label = np.unique(labels)
print(labels)
print(label)
# label_to_index = dict((item, index) for (index, item) in enumerate(label))
# index_to_label = dict((index, item) for (item, index) in label_to_index.items())
 
# index_labels = [label_to_index.get(name) for name in labels] # 映射
# print(index_label)

In [ ]:
#index_labels[:3] # index_labels为标签数组

In [ ]:
random_index = np.random.permutation(len(img_path))
 
img_path = np.array(img_path)[random_index]
label_set = np.array(label_set)[random_index]
# print(train_img_path)
divider = int(len(img_path) * 0.8)
# print(divider)
train_path = img_path[:divider]
train_label = label_set[:divider]
test_path = img_path[divider:]
test_label = label_set[divider:]
# print(train_label)
# print(train_path)
# print(test_path)

In [ ]:
# 数据集
# print(train_path)

train_datasets = tf.data.Dataset.from_tensor_slices((train_path, train_label))

for element in train_datasets:
  print(element)
test_datasets = tf.data.Dataset.from_tensor_slices((test_path, test_label))

In [ ]:
def load_img(img_path, img_label):
    image = tf.io.read_file(img_path)
    image = tf.image.decode_png(image, channels = 3, dtype=tf.dtypes.uint8) # 将image解码，通道数为3
    #image = tf.image.decode_jpeg(image, channels = 3) # 将image解码，通道数为3
    image = tf.image.resize(image, [224, 224])
    image = tf.cast(image, tf.float32)
    image = image / 255 # 归一化
    return image, img_label
img11, img_label11 = load_img(train_path[1], train_label[1])

AUTOTUNE = tf.data.experimental.AUTOTUNE # 多线程
train_datasets = train_datasets.map(load_img, num_parallel_calls=AUTOTUNE)
test_datasets = test_datasets.map(load_img, num_parallel_calls=AUTOTUNE)
BATCH_SIZE = 2

In [ ]:
print(img11)
print(img_label11)


In [ ]:
train_datasets = train_datasets.repeat().shuffle(300).batch(BATCH_SIZE)
# <BatchDataset shapes: ((None, 224, 224, 3), (None,)), types: (tf.float32, tf.int64)>
#print(train_datasets.size)
test_datasets = test_datasets.batch(BATCH_SIZE)

# <BatchDataset shapes: ((None, 224, 224, 3), (None,)), types: (tf.float32, tf.int64)>

# ***The code template for residual unit***

In [ ]:
 
def residual_unit(img_data, channel_of_filter, strides=(2, 2)):
 
    channel1, channel2, channel3 = channel_of_filter
    #bottelneck

    # the first 1*1 conv
    x = Conv2D(channel1, (1, 1), strides=strides,)(img_data)
    x = BatchNormalization()(x)
    x = Dropout(0.2)(x) # dropout
    x = Activation('relu')(x)
 
    #the middle 3*3 conv  
    x = Conv2D(channel2, (3, 3), padding='same')(x)
    x = BatchNormalization()(x)
    x = Dropout(0.2)(x)
    x = Activation('relu')(x)
 
    # the last 1*1 conv
    x = Conv2D(channel3, (1, 1))(x)
    x = BatchNormalization()(x)
    x = Dropout(0.2)(x) # dropout
 
    # shortcur settings
    shortcut = Conv2D(channel3, (1, 1), strides=strides)(img_data)
    shortcut = BatchNormalization()(shortcut)
    x = layers.add([x, shortcut])  #'add' algorithm
    x = Activation('relu')(x)


    return x

def residual_unit2(img_data, channel_of_filter, strides=(2, 2)):
 
    channel1, channel2, channel3 = channel_of_filter
    #bottelneck

    # the first 1*1 conv
    x = Conv2D(channel1, (1, 1), strides=strides,)(img_data)
    x = BatchNormalization()(x)
    x = Dropout(0.2)(x) # dropout
    x = Activation('relu')(x)
 
    #the middle 3*3 conv  
    x = Conv2D(channel2, (3, 3), padding='same')(x)
    x = BatchNormalization()(x)
    x = Dropout(0.2)(x)
    x = Activation('relu')(x)
 
    # the last 1*1 conv
    x = Conv2D(channel3, (1, 1))(x)
    x = BatchNormalization()(x)
    x = Dropout(0.2)(x) # dropout
 
    
    x = layers.add([x, img_data])  #'add' algorithm
    x = Activation('relu')(x)


    return x




# ***code higgen***

In [ ]:
 
def conv_block(img_data, kernel_size, filters, stage, block, strides=(2, 2)):
 
    # 64,64,256
    filters1, filters2, filters3 = filters
 
    conv_name_base = 'res' + str(stage) + block + '_branch'
    bn_name_base = 'bn' + str(stage) + block + '_branch'
 
    # the first 1*1 conv
    x = Conv2D(filters1, (1, 1), strides=strides,
               name=conv_name_base + '2a')(img_data)
    x = BatchNormalization(name=bn_name_base + '2a')(x)
    x = Dropout(0.2)(x) # dropout
    x = Activation('relu')(x)
 
    # 
    x = Conv2D(filters2, kernel_size, padding='same',
               name=conv_name_base + '2b')(x)
    x = BatchNormalization(name=bn_name_base + '2b')(x)
    
    # dropout
    x = Dropout(0.2)(x)
    x = Activation('relu')(x)
 
    # 升维
    x = Conv2D(filters3, (1, 1), name=conv_name_base + '2c')(x)
    x = BatchNormalization(name=bn_name_base + '2c')(x)
    x = Dropout(0.2)(x) # dropout
 
    # 残差边
    shortcut = Conv2D(filters3, (1, 1), strides=strides, name=conv_name_base + '1')(img_data) # 将input_tensor转换为对应维度（w x h x c）
    shortcut = BatchNormalization(name=bn_name_base + '1')(shortcut)
 
 
    x = layers.add([x, shortcut])
    x = Activation('relu')(x)
    return x

In [ ]:
def identity_block(input_tensor, kernel_size, filters, stage, block):
 
    [filters1, filters2, filters3] = filters
    # filters = [512, 512, 1024]
    # filters1 = 512, filters2 = 512, filters3 = 1024
    # print(filters1)
    # print(filters)
 
    conv_name_base = 'res' + str(stage) + block + '_branch'
    bn_name_base = 'bn' + str(stage) + block + '_branch'
 
    # 降维
    x = Conv2D(filters1, (1, 1), name=conv_name_base + '2a')(input_tensor)
    x = BatchNormalization(name=bn_name_base + '2a')(x)
    x = Dropout(0.2)(x) # dropout
    x = Activation('relu')(x)
    
    
    # 3x3卷积
    x = Conv2D(filters2, kernel_size,padding='same', name=conv_name_base + '2b')(x)
    
    x = Dropout(0.2)(x) # dropout
    
    x = BatchNormalization(name=bn_name_base + '2b')(x)
    x = Activation('relu')(x)
    
    
    # 升维
    x = Conv2D(filters3, (1, 1), name=conv_name_base + '2c')(x)
    x = BatchNormalization(name=bn_name_base + '2c')(x)
    x = Dropout(0.2)(x) # dropout
 
    x = layers.add([x, input_tensor])
    x = Activation('relu')(x)
    return x

# ***MODIFIED SHAPE DATA FOR THE RESNET*** 


In [ ]:
def conv_block(img_data, kernel_size, filters, stage, block, strides=(2, 2)):
 
    # 64,64,256
    filters1, filters2, filters3 = filters
 
    conv_name_base = 'res' + str(stage) + block + '_branch'
    bn_name_base = 'bn' + str(stage) + block + '_branch'
 
    # the first 1*1 conv
    x = Conv2D(filters1, (1, 1), strides=strides,
               name=conv_name_base + '2a')(img_data)
    x = BatchNormalization(name=bn_name_base + '2a')(x)
    x = Dropout(0.2)(x) # dropout
    x = Activation('relu')(x)
 
    # 
    x = Conv2D(filters2, kernel_size, padding='same',
               name=conv_name_base + '2b')(x)
    x = BatchNormalization(name=bn_name_base + '2b')(x)
    
    # dropout
    x = Dropout(0.2)(x)
    x = Activation('relu')(x)
 
    # 升维
    x = Conv2D(filters3, (1, 1), name=conv_name_base + '2c')(x)
    x = BatchNormalization(name=bn_name_base + '2c')(x)
    x = Dropout(0.2)(x) # dropout
 
    # 残差边
    shortcut = Conv2D(filters3, (1, 1), strides=strides, name=conv_name_base + '1')(img_data) # 将input_tensor转换为对应维度（w x h x c）
    shortcut = BatchNormalization(name=bn_name_base + '1')(shortcut)
 
 
    x = layers.add([x, shortcut])
    x = Activation('relu')(x)
    return x





  def identity_block(input_tensor, kernel_size, filters, stage, block):
 
    [filters1, filters2, filters3] = filters
    # filters = [512, 512, 1024]
    # filters1 = 512, filters2 = 512, filters3 = 1024
    # print(filters1)
    # print(filters)
 
    conv_name_base = 'res' + str(stage) + block + '_branch'
    bn_name_base = 'bn' + str(stage) + block + '_branch'
 
    # 降维
    x = Conv2D(filters1, (1, 1), name=conv_name_base + '2a')(input_tensor)
    x = BatchNormalization(name=bn_name_base + '2a')(x)
    x = Dropout(0.2)(x) # dropout
    x = Activation('relu')(x)
    
    
    # 3x3卷积
    x = Conv2D(filters2, kernel_size,padding='same', name=conv_name_base + '2b')(x)
    
    x = Dropout(0.2)(x) # dropout
    
    x = BatchNormalization(name=bn_name_base + '2b')(x)
    x = Activation('relu')(x)
    
    
    # 升维
    x = Conv2D(filters3, (1, 1), name=conv_name_base + '2c')(x)
    x = BatchNormalization(name=bn_name_base + '2c')(x)
    x = Dropout(0.2)(x) # dropout
 
    x = layers.add([x, input_tensor])
    x = Activation('relu')(x)
    return x

# **The code template for Resnet50**

In [ ]:
def ResNet50(img_size=[224,224,3],classes=5):

    img_input = tf.keras.layers.Input(shape=img_size)
    x = ZeroPadding2D((3, 3))(img_input)   

    #stage 1
    x = Conv2D(64, (7, 7), strides=(2, 2))(x)  
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
 
    x = MaxPooling2D((3, 3), strides=(2, 2))(x) 
    
    #[filter1, filter2 , filter3]=[64, 64, 256]
 
    # stage 2
    x = residual_unit(x, [64, 64, 256], strides=(1, 1))
    x = residual_unit2(x,  [64, 64, 256])
    x = residual_unit2(x,  [64, 64, 256])
 
    # stage 3
    x = residual_unit(x, [128, 128, 512])
    x = residual_unit2(x, [128, 128, 512])
    x = residual_unit2(x, [128, 128, 512])
    x = residual_unit2(x, [128, 128, 512])
 
    # stage 4
    x = residual_unit(x, [256, 256, 1024])
    x = residual_unit2(x, [256, 256, 1024])
    x = residual_unit2(x, [256, 256, 1024])
    x = residual_unit2(x, [256, 256, 1024])
    x = residual_unit2(x, [256, 256, 1024])
    x = residual_unit2(x, [256, 256, 1024])
   
 
    # stage 5
    x = residual_unit(x, [512, 512, 2048])
    x = residual_unit2(x, [512, 512, 2048])
    x = residual_unit2(x, [512, 512, 2048])
 
    # the last layer
    x = AveragePooling2D((7, 7), name='avg_pool')(x)
    x = Flatten()(x)
    x = Dense(classes, activation='softmax', name='fc5')(x)

    #create the model
    model = Model(img_input, x, name='resnet50')
 
    return model

# ***The real code for the module ***

In [ ]:

def ResNet50(input_shape=[224,224,3],classes=5):
    # [224,224,3]
    img_input = tf.keras.layers.Input(shape=input_shape)
    x = ZeroPadding2D((3, 3))(img_input)   # [230,230,3]
    # [112,112,64]


    #15*15
    #filter, under this condtion, is the cahnnel number 
    x = Conv2D(64, (7, 7), strides=(2, 2), name='conv1')(x)   #[112,112,64]   [15 15 64]
    x = BatchNormalization(name='bn_conv1')(x)
    x = Activation('relu')(x)
 
    # [56,56,64]   [48 48 64]
    #x = MaxPooling2D((3, 3), strides=(2, 2))(x)  #channel 128
    x = MaxPooling2D((2, 2), strides=(2, 2))(x)  #channel 128
 
    # [56,56,256]
    x = conv_block(x, 3, [64, 64, 256], stage=2, block='a', strides=(1, 1))
    x = identity_block(x, 3, [64, 64, 256], stage=2, block='b')
    x = identity_block(x, 3, [64, 64, 256], stage=2, block='c')
 
    # [28,28,512]
    x = conv_block(x, 3, [128, 128, 512], stage=3, block='a')
    x = identity_block(x, 3, [128, 128, 512], stage=3, block='b')
    x = identity_block(x, 3, [128, 128, 512], stage=3, block='c')
    x = identity_block(x, 3, [128, 128, 512], stage=3, block='d')
 
    # [14,14,1024]
    x = conv_block(x, 3, [256, 256, 1024], stage=4, block='a')
    x = identity_block(x, 3, [256, 256, 1024], stage=4, block='b')
    x = identity_block(x, 3, [256, 256, 1024], stage=4, block='c')
    x = identity_block(x, 3, [256, 256, 1024], stage=4, block='d')
    x = identity_block(x, 3, [256, 256, 1024], stage=4, block='e')
    x = identity_block(x, 3, [256, 256, 1024], stage=4, block='f')
   
 
    # [7,7,2048]
    x = conv_block(x, 3, [512, 512, 2048], stage=5, block='a')
    x = identity_block(x, 3, [512, 512, 2048], stage=5, block='b')
    x = identity_block(x, 3, [512, 512, 2048], stage=5, block='c')
 
    # 代替全连接层
    x = AveragePooling2D((7, 7), name='avg_pool')(x)
 
    # 进行预测
    x = Flatten()(x)
    x = Dense(classes, activation='softmax', name='fc5')(x)
 
    model = Model(img_input, x, name='resnet50')
 
    return model

# ***MODIFIED MDOEL FOR DIFERENT SHAPE OF THE INPUT*** 


In [ ]:
def ResNet50(input_shape=[100 , 100 ,3],classes=5):
    # [224,224,3]
    img_input = tf.keras.layers.Input(shape=input_shape)
    x = ZeroPadding2D((3 , 3))(img_input)   # [230,230,3]
    # [112,112,64]


    #15*15
    #filter, under this condtion, is the cahnnel number 
    x = Conv2D(64, (2 , 2 ), strides=(2, 2), name='conv1')(x)   #[112,112,64]   [15 15 64]
    x = BatchNormalization(name='bn_conv1')(x)
    x = Activation('relu')(x)
 
    # [56,56,64]   [48 48 64]
    #x = MaxPooling2D((3, 3), strides=(2, 2))(x)  #channel 128
    x = MaxPooling2D((2, 2), strides=(2, 2))(x)  #channel 128
 
    # [56,56,256]
    x = conv_block(x, 3, [64, 64, 256], stage=2, block='a', strides=(1, 1))
    x = identity_block(x, 3, [64, 64, 256], stage=2, block='b')
    x = identity_block(x, 3, [64, 64, 256], stage=2, block='c')
 
    # [28,28,512]
    x = conv_block(x, 3, [128, 128, 512], stage=3, block='a')
    x = identity_block(x, 3, [128, 128, 512], stage=3, block='b')
    x = identity_block(x, 3, [128, 128, 512], stage=3, block='c')
    x = identity_block(x, 3, [128, 128, 512], stage=3, block='d')
 
    # [14,14,1024]
    x = conv_block(x, 3, [256, 256, 1024], stage=4, block='a')
    x = identity_block(x, 3, [256, 256, 1024], stage=4, block='b')
    x = identity_block(x, 3, [256, 256, 1024], stage=4, block='c')
    x = identity_block(x, 3, [256, 256, 1024], stage=4, block='d')
    x = identity_block(x, 3, [256, 256, 1024], stage=4, block='e')
    x = identity_block(x, 3, [256, 256, 1024], stage=4, block='f')
   
 
    # [7,7,2048]
    x = conv_block(x, 3, [512, 512, 2048], stage=5, block='a')
    x = identity_block(x, 3, [512, 512, 2048], stage=5, block='b')
    x = identity_block(x, 3, [512, 512, 2048], stage=5, block='c')
 
    # 代替全连接层
    x = AveragePooling2D((3 , 3), name='avg_pool')(x)
 
    # 进行预测
    x = Flatten()(x)
    x = Dense(classes, activation='softmax', name='fc5')(x)
 
    model = Model(img_input, x, name='resnet50')
 
    return model

# ***Use this one with different shape***

In [ ]:
# resnet18： ResNet(BasicBlock, [2, 2, 2, 2])
 
# resnet34： ResNet(BasicBlock, [3, 4, 6, 3])
 
# resnet50：ResNet(Bottleneck, [3, 4, 6, 3])
 
# resnet101:ResNet(Bottleneck, [3, 4, 23, 3])
 
# resnet152:ResNet(Bottleneck, [3, 8, 36, 3])
 
def ResNet50(input_shape=[30, 30 ,3],classes=5):
    # [224,224,3]
    img_input = tf.keras.layers.Input(shape=input_shape)
    x = ZeroPadding2D((3, 3))(img_input)   # [230,230,3]
    # [112,112,64]
    #filter, under this condtion, is the cahnnel number 
    x = Conv2D(64, (7, 7), strides=(2, 2), name='conv1')(x)   #[112,112,64]
    x = BatchNormalization(name='bn_conv1')(x)
    x = Activation('relu')(x)
 
    # [56,56,64]
    x = MaxPooling2D((3, 3), strides=(2, 2))(x)
 
    # [56,56,256]
    x = conv_block(x, 3, [64, 64, 256], stage=2, block='a', strides=(1, 1))
    x = identity_block(x, 3, [64, 64, 256], stage=2, block='b')
    x = identity_block(x, 3, [64, 64, 256], stage=2, block='c')
 
    # [28,28,512]
    x = conv_block(x, 3, [128, 128, 512], stage=3, block='a')
    x = identity_block(x, 3, [128, 128, 512], stage=3, block='b')
    x = identity_block(x, 3, [128, 128, 512], stage=3, block='c')
    x = identity_block(x, 3, [128, 128, 512], stage=3, block='d')
 
    # [14,14,1024]
    x = conv_block(x, 3, [256, 256, 1024], stage=4, block='a')
    x = identity_block(x, 3, [256, 256, 1024], stage=4, block='b')
    x = identity_block(x, 3, [256, 256, 1024], stage=4, block='c')
    x = identity_block(x, 3, [256, 256, 1024], stage=4, block='d')
    x = identity_block(x, 3, [256, 256, 1024], stage=4, block='e')
    x = identity_block(x, 3, [256, 256, 1024], stage=4, block='f')
   
 
    # [7,7,2048]
    x = conv_block(x, 3, [512, 512, 2048], stage=5, block='a')
    x = identity_block(x, 3, [512, 512, 2048], stage=5, block='b')
    x = identity_block(x, 3, [512, 512, 2048], stage=5, block='c')
 
    # 代替全连接层
    x = AveragePooling2D((7, 7), name='avg_pool')(x)
 
    # 进行预测
    x = Flatten()(x)
    x = Dense(classes, activation='softmax', name='fc5')(x)
 
    model = Model(img_input, x, name='resnet50')
 
    return model

hide the code
# ***New section***
```
# This is formatted as code
```



In [ ]:
def ResNet101(input_shape=[224,224,3],classes=5):
    # [224,224,3]
    img_input = tf.keras.layers.Input(shape=input_shape)
    x = ZeroPadding2D((3, 3))(img_input)   # [230,230,3]
    # [112,112,64]
    x = Conv2D(64, (7, 7), strides=(2, 2), name='conv1')(x)   #[112,112,64]
    x = BatchNormalization(name='bn_conv1')(x)
    x = Activation('relu')(x)
 
    # [56,56,64]
    x = MaxPooling2D((3, 3), strides=(2, 2))(x)
 
    # [56,56,256]
    x = conv_block(x, 3, [64, 64, 256], stage=2, block='a', strides=(1, 1))
    x = identity_block(x, 3, [64, 64, 256], stage=2, block='b')
    x = identity_block(x, 3, [64, 64, 256], stage=2, block='c')
 
    # [28,28,512]
    x = conv_block(x, 3, [128, 128, 512], stage=3, block='a')
    x = identity_block(x, 3, [128, 128, 512], stage=3, block='b')
    x = identity_block(x, 3, [128, 128, 512], stage=3, block='c')
    x = identity_block(x, 3, [128, 128, 512], stage=3, block='d')
 
    # [14,14,1024]
    x = conv_block(x, 3, [256, 256, 1024], stage=4, block='a')
    x = identity_block(x, 3, [256, 256, 1024], stage=4, block='b')
    x = identity_block(x, 3, [256, 256, 1024], stage=4, block='c')
    x = identity_block(x, 3, [256, 256, 1024], stage=4, block='d')
    x = identity_block(x, 3, [256, 256, 1024], stage=4, block='e')
    x = identity_block(x, 3, [256, 256, 1024], stage=4, block='f')
    x = identity_block(x, 3, [256, 256, 1024], stage=4, block='g')
    x = identity_block(x, 3, [256, 256, 1024], stage=4, block='h')
    x = identity_block(x, 3, [256, 256, 1024], stage=4, block='i')
    x = identity_block(x, 3, [256, 256, 1024], stage=4, block='j')
    x = identity_block(x, 3, [256, 256, 1024], stage=4, block='k')
    x = identity_block(x, 3, [256, 256, 1024], stage=4, block='l')
    x = identity_block(x, 3, [256, 256, 1024], stage=4, block='m')
    x = identity_block(x, 3, [256, 256, 1024], stage=4, block='n')
    x = identity_block(x, 3, [256, 256, 1024], stage=4, block='o')
    x = identity_block(x, 3, [256, 256, 1024], stage=4, block='p')
    x = identity_block(x, 3, [256, 256, 1024], stage=4, block='q')
    x = identity_block(x, 3, [256, 256, 1024], stage=4, block='r')
    x = identity_block(x, 3, [256, 256, 1024], stage=4, block='s')
    x = identity_block(x, 3, [256, 256, 1024], stage=4, block='t')
    x = identity_block(x, 3, [256, 256, 1024], stage=4, block='u')
    x = identity_block(x, 3, [256, 256, 1024], stage=4, block='v')
    x = identity_block(x, 3, [256, 256, 1024], stage=4, block='w')
 
    # [7,7,2048]
    x = conv_block(x, 3, [512, 512, 2048], stage=5, block='a')
    x = identity_block(x, 3, [512, 512, 2048], stage=5, block='b')
    x = identity_block(x, 3, [512, 512, 2048], stage=5, block='c')
 
    # 代替全连接层
    x = AveragePooling2D((7, 7), name='avg_pool')(x)
 
    # 分类预测
    x = Flatten()(x)
    x = Dense(classes, activation='softmax', name='fc5')(x)
 
    model = Model(img_input, x, name='resnet101')
 
    return model

# ***Block***

In [ ]:
# model = ResNet50(input_shape=[224,224,3],classes=5)
model = ResNet50()
#model = ResNet101()

In [ ]:
sgd = tf.keras.optimizers.SGD(lr=0.01, momentum=0.8, decay=0.0, nesterov=False)
BATCH_SIZE = 100
model.compile(optimizer=sgd,
             #loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=False),
             loss = 'sparse_categorical_crossentropy', #交叉熵损失
             metrics=['accuracy']
             )
 
# train_count = len(train_path)
# test_count = len(test_path)
 
steps_per_epoch = spl // BATCH_SIZE
validation_steps = (num_limit-spl) // BATCH_SIZE

In [ ]:
from tensorflow.keras.callbacks import EarlyStopping
 
early_stopping = EarlyStopping(
    min_delta=0.0008, # minimium amount of change to count as an improvement
    patience=40, # how many epochs to wait before stopping
    restore_best_weights=True,
)

In [ ]:
tf.config.run_functions_eagerly(True)

In [ ]:
tf.config.run_functions_eagerly(True)


# 开始训
# for element in test_datasets:
#   #print(element)
#   history = model.fit(
#                     input_tensor, label_tensor,
#                     epochs = 120,
#                     steps_per_epoch = steps_per_epoch,
#                     validation_data = test_datasets,
#                     validation_steps = validation_steps,
#                     callbacks=[early_stopping], # put your callbacks in a list
#                     )
history = model.fit(input_tensor, label_tensor,
                    epochs = 10 ,
                    steps_per_epoch = steps_per_epoch,
                    validation_data = (test_input_tensor,test_label_tensor),
                    validation_steps = validation_steps,
                    callbacks=[early_stopping], # put your callbacks in a list
                    )

In [ ]:
#model.save('myModel.h5')

history.history.keys()
# dict_keys(['loss', 'accuracy', 'val_loss', 'val_accuracy'])

In [ ]:
# 画出精确值变化图像
plt.plot(history.epoch, history.history.get('accuracy'), label='accuracy')
plt.plot(history.epoch, history.history.get('val_accuracy'), label='val_accuracy')
plt.legend()

In [ ]:
# 画出损失值变化图像
plt.plot(history.epoch, history.history.get('loss'), label='loss')
plt.plot(history.epoch, history.history.get('val_loss'), label='val_loss')
plt.legend()